# Predicting CitiBike Demand

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

## Part 1: Data Loading and Preprocessing

### Load Data

In [16]:
#df_train = pd.read_csv('data/2024_08/df_train.csv', low_memory=False)
df_weather = pd.read_csv('data/2024_08/weather_data_2024.csv', low_memory=False)

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603575 entries, 0 to 4603574
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 456.6+ MB


In [4]:
df_train.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB4E29A011E918E,electric_bike,2024-08-13 22:28:13.065,2024-08-13 22:30:43.138,McKibbin St & Bogart St,5059.02,Wilson Ave & Troutman St,4864.09,40.706237,-73.933871,40.701660,-73.927540,member
1,6389E1E171CE17CD,classic_bike,2024-08-07 09:39:52.489,2024-08-07 09:43:14.975,Bialystoker Pl & Delancey St,5335.03,Norfolk St & Broome St,5374.01,40.716226,-73.982612,40.717227,-73.988021,member
2,3F4BBEBDFB7548C0,electric_bike,2024-08-10 21:04:35.143,2024-08-10 21:21:43.846,Rivington St & Chrystie St,5453.01,Kent Ave & Division Ave,5021.05,40.721101,-73.991925,40.706564,-73.968319,member
3,C0939F0CD7ED731E,classic_bike,2024-08-13 19:21:31.275,2024-08-13 19:33:43.790,Broadway & E 21 St,6098.10,1 Ave & E 39 St,6303.01,40.739888,-73.989586,40.747140,-73.971130,member
4,4CB3950095D804D6,electric_bike,2024-08-09 22:23:42.894,2024-08-09 22:58:04.455,E 34 St & Church Ave,3318.05,48 St & 2 Ave,3283.05,40.651160,-73.945770,40.650176,-74.015606,casual
5,585D1A3FEEF4867E,electric_bike,2024-08-02 16:58:20.753,2024-08-02 17:03:15.327,McKibbin St & Bogart St,5059.02,Suydam St & Broadway,4689.03,40.706237,-73.933871,40.695440,-73.932230,member
6,BAC50AFB465C607B,electric_bike,2024-08-07 23:12:24.982,2024-08-07 23:20:00.458,Broadway & E 21 St,6098.10,1 Ave & E 39 St,6303.01,40.739888,-73.989586,40.747140,-73.971130,member
7,4F4D162103E66917,electric_bike,2024-08-06 15:20:31.886,2024-08-06 15:32:23.192,8 Ave & W 16 St,6072.11,1 Ave & E 39 St,6303.01,40.740983,-74.001702,40.747140,-73.971130,member
8,DE6F03D235645CBF,classic_bike,2024-08-13 18:51:17.796,2024-08-13 19:06:27.537,Graham Ave & Grand St,5178.06,Stanton St & Norfolk St,5445.07,40.711863,-73.944024,40.720747,-73.986274,member
9,BC3A1D7C8884C727,electric_bike,2024-08-05 00:24:16.628,2024-08-05 00:58:38.898,8 Ave & W 16 St,6072.11,5 Ave & W 131 St,7735.05,40.740983,-74.001702,40.810140,-73.939730,member


In [5]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              768 non-null    int64  
 1   datetime          768 non-null    object 
 2   temp              768 non-null    float64
 3   feelslike         768 non-null    float64
 4   dew               768 non-null    float64
 5   humidity          768 non-null    float64
 6   precip            768 non-null    float64
 7   precipprob        768 non-null    int64  
 8   preciptype        78 non-null     object 
 9   snow              768 non-null    int64  
 10  snowdepth         768 non-null    int64  
 11  windgust          768 non-null    float64
 12  windspeed         768 non-null    float64
 13  winddir           768 non-null    float64
 14  sealevelpressure  768 non-null    float64
 15  cloudcover        768 non-null    float64
 16  visibility        768 non-null    float64
 1

In [6]:
df_weather.head(10)

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,10021,2024-07-31T00:00:00,77.0,77.0,70.1,79.36,0.0,0,NaN,0,...,1013.9,99.8,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
1,10021,2024-07-31T01:00:00,77.0,77.0,70.1,79.40,0.0,0,NaN,0,...,1013.3,89.1,9.9,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
2,10021,2024-07-31T02:00:00,77.0,77.0,71.1,81.90,0.0,0,NaN,0,...,1013.1,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
3,10021,2024-07-31T03:00:00,77.0,77.0,71.1,82.06,0.0,0,NaN,0,...,1012.7,90.0,9.9,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
4,10021,2024-07-31T04:00:00,77.0,77.0,71.1,82.10,0.0,0,NaN,0,...,1012.1,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
5,10021,2024-07-31T05:00:00,76.9,76.9,71.1,82.30,0.0,0,NaN,0,...,1012.2,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
6,10021,2024-07-31T06:00:00,77.0,77.0,72.0,84.43,0.0,0,NaN,0,...,1012.6,100.0,9.9,15,0.1,0,NaN,Overcast,cloudy,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"
7,10021,2024-07-31T07:00:00,78.1,78.1,71.1,79.21,0.0,0,NaN,0,...,1012.4,89.8,9.9,83,0.3,1,NaN,Partially cloudy,partly-cloudy-day,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"
8,10021,2024-07-31T08:00:00,78.2,78.2,71.1,78.83,0.0,0,NaN,0,...,1012.3,99.3,9.9,104,0.4,1,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
9,10021,2024-07-31T09:00:00,78.3,78.3,70.1,75.94,0.0,0,NaN,0,...,1012.6,99.3,9.9,139,0.5,1,NaN,Overcast,cloudy,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"


### Clean and Prepare Data

In [10]:
df_train.head(-10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB4E29A011E918E,electric_bike,2024-08-13 22:28:13.065,2024-08-13 22:30:43.138,McKibbin St & Bogart St,5059.02,Wilson Ave & Troutman St,4864.09,40.706237,-73.933871,40.701660,-73.927540,member
1,6389E1E171CE17CD,classic_bike,2024-08-07 09:39:52.489,2024-08-07 09:43:14.975,Bialystoker Pl & Delancey St,5335.03,Norfolk St & Broome St,5374.01,40.716226,-73.982612,40.717227,-73.988021,member
2,3F4BBEBDFB7548C0,electric_bike,2024-08-10 21:04:35.143,2024-08-10 21:21:43.846,Rivington St & Chrystie St,5453.01,Kent Ave & Division Ave,5021.05,40.721101,-73.991925,40.706564,-73.968319,member
3,C0939F0CD7ED731E,classic_bike,2024-08-13 19:21:31.275,2024-08-13 19:33:43.790,Broadway & E 21 St,6098.10,1 Ave & E 39 St,6303.01,40.739888,-73.989586,40.747140,-73.971130,member
4,4CB3950095D804D6,electric_bike,2024-08-09 22:23:42.894,2024-08-09 22:58:04.455,E 34 St & Church Ave,3318.05,48 St & 2 Ave,3283.05,40.651160,-73.945770,40.650176,-74.015606,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603560,38FD7D8E16816AC9,electric_bike,2024-08-31 15:35:53.969,2024-08-31 16:12:00.649,Henry St & Atlantic Ave,4531.05,N 12 St & Bedford Ave,5450.04,40.690893,-73.996123,40.720798,-73.954847,member
4603561,3BDAD1143141FAEF,electric_bike,2024-08-21 19:45:27.492,2024-08-21 19:56:25.075,Berkeley Pl & 7 Ave,4051.01,Parkside Ave & Parade Pl,3376.04,40.675147,-73.975232,40.653200,-73.966390,member
4603562,E3CD0013606E72AF,electric_bike,2024-08-17 16:16:10.281,2024-08-17 16:30:25.909,6 Ave & Canal St,5500.07,4 Ave & E 12 St,5788.15,40.722438,-74.005664,40.732647,-73.990110,casual
4603563,893D7B066F924BBC,electric_bike,2024-08-16 18:38:31.132,2024-08-16 19:04:34.490,Henry St & Atlantic Ave,4531.05,Parkside Ave & Parade Pl,3376.04,40.690893,-73.996123,40.653200,-73.966390,casual


In [11]:
# Check for duplicates
duplicates = df_train.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


In [12]:
# Check for missing values
missing_values = df_train.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name     2962
start_station_id       2962
end_station_name      12410
end_station_id        13353
start_lat              2962
start_lng              2962
end_lat               13332
end_lng               13332
member_casual             0
dtype: int64


In [13]:
# Compute percentage of missing values for each column
missing_percentage = (df_train.isnull().sum() / len(df_train)) * 100
print("Percentage of missing values in each column:")
print(missing_percentage)

Percentage of missing values in each column:
ride_id               0.000000
rideable_type         0.000000
started_at            0.000000
ended_at              0.000000
start_station_name    0.064341
start_station_id      0.064341
end_station_name      0.269573
end_station_id        0.290057
start_lat             0.064341
start_lng             0.064341
end_lat               0.289601
end_lng               0.289601
member_casual         0.000000
dtype: float64


In [14]:
# Drop rows with missing values where & missing < 5%
threshold = 5.0
cols_to_check = missing_percentage[missing_percentage < threshold].index
df_train = df_train.dropna(subset=cols_to_check)            


In [15]:
# Check results
missing_values = df_train.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64


#### Borough Identification

In [16]:
# Extrapolate borough boundaries
from shapely.geometry import Point
from shapely import wkt

# Load the borough boundaries CSV
borough_df = pd.read_csv('data/2024_08/Borough_Boundaries_20251104.csv')

# Parse the geometry column (it's in WKT format)
boroughs = {}
for _, row in borough_df.iterrows():
    borough_name = row['BoroName']
    geometry = wkt.loads(row['the_geom'])  # Convert WKT string to Shapely geometry
    boroughs[borough_name] = geometry

print(f"Loaded {len(boroughs)} boroughs: {list(boroughs.keys())}")


Loaded 5 boroughs: ['Staten Island', 'Bronx', 'Brooklyn', 'Queens', 'Manhattan']


In [17]:
# Function to find which borough a point is in
def get_borough(lat, lng):
    point = Point(lng, lat)  # Note: Point takes (longitude, latitude)
    for borough_name, geometry in boroughs.items():
        if geometry.contains(point):
            return borough_name
    return 'Unknown'

In [18]:
# Execute the function for Start Locations
df_train['start_borough'] = df_train.apply(
    lambda row: get_borough(row['start_lat'], row['start_lng']), 
    axis=1
)

print("\nBorough distribution:")
print(df_train['start_borough'].value_counts())


Borough distribution:
Manhattan    2847392
Brooklyn     1263642
Queens        327405
Bronx         149429
Name: start_borough, dtype: int64


In [19]:
# Execute the function for End Locations
df_train['end_borough'] = df_train.apply(
    lambda row: get_borough(row['end_lat'], row['end_lng']), 
    axis=1
)

print("\nEnd borough distribution:")
print(df_train['end_borough'].value_counts())


End borough distribution:
Manhattan    2840304
Brooklyn     1270190
Queens        326190
Bronx         150848
Unknown          336
Name: end_borough, dtype: int64


In [20]:
df_train.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_borough,end_borough
0,5CB4E29A011E918E,electric_bike,2024-08-13 22:28:13.065,2024-08-13 22:30:43.138,McKibbin St & Bogart St,5059.02,Wilson Ave & Troutman St,4864.09,40.706237,-73.933871,40.701660,-73.927540,member,Brooklyn,Brooklyn
1,6389E1E171CE17CD,classic_bike,2024-08-07 09:39:52.489,2024-08-07 09:43:14.975,Bialystoker Pl & Delancey St,5335.03,Norfolk St & Broome St,5374.01,40.716226,-73.982612,40.717227,-73.988021,member,Manhattan,Manhattan
2,3F4BBEBDFB7548C0,electric_bike,2024-08-10 21:04:35.143,2024-08-10 21:21:43.846,Rivington St & Chrystie St,5453.01,Kent Ave & Division Ave,5021.05,40.721101,-73.991925,40.706564,-73.968319,member,Manhattan,Brooklyn
3,C0939F0CD7ED731E,classic_bike,2024-08-13 19:21:31.275,2024-08-13 19:33:43.790,Broadway & E 21 St,6098.10,1 Ave & E 39 St,6303.01,40.739888,-73.989586,40.747140,-73.971130,member,Manhattan,Manhattan
4,4CB3950095D804D6,electric_bike,2024-08-09 22:23:42.894,2024-08-09 22:58:04.455,E 34 St & Church Ave,3318.05,48 St & 2 Ave,3283.05,40.651160,-73.945770,40.650176,-74.015606,casual,Brooklyn,Brooklyn
5,585D1A3FEEF4867E,electric_bike,2024-08-02 16:58:20.753,2024-08-02 17:03:15.327,McKibbin St & Bogart St,5059.02,Suydam St & Broadway,4689.03,40.706237,-73.933871,40.695440,-73.932230,member,Brooklyn,Brooklyn
6,BAC50AFB465C607B,electric_bike,2024-08-07 23:12:24.982,2024-08-07 23:20:00.458,Broadway & E 21 St,6098.10,1 Ave & E 39 St,6303.01,40.739888,-73.989586,40.747140,-73.971130,member,Manhattan,Manhattan
7,4F4D162103E66917,electric_bike,2024-08-06 15:20:31.886,2024-08-06 15:32:23.192,8 Ave & W 16 St,6072.11,1 Ave & E 39 St,6303.01,40.740983,-74.001702,40.747140,-73.971130,member,Manhattan,Manhattan
8,DE6F03D235645CBF,classic_bike,2024-08-13 18:51:17.796,2024-08-13 19:06:27.537,Graham Ave & Grand St,5178.06,Stanton St & Norfolk St,5445.07,40.711863,-73.944024,40.720747,-73.986274,member,Brooklyn,Manhattan
9,BC3A1D7C8884C727,electric_bike,2024-08-05 00:24:16.628,2024-08-05 00:58:38.898,8 Ave & W 16 St,6072.11,5 Ave & W 131 St,7735.05,40.740983,-74.001702,40.810140,-73.939730,member,Manhattan,Manhattan


In [21]:
# Keep only Manhattan, Brooklyn Start and End Boroughs
df_train = df_train[df_train['start_borough'].isin(['Manhattan', 'Brooklyn'])]
df_train = df_train[df_train['end_borough'].isin(['Manhattan', 'Brooklyn'])]

In [22]:
# Check results after dropping
print("\nBorough distribution:")
print(df_train['start_borough'].value_counts())

print("\nEnd borough distribution:")
print(df_train['end_borough'].value_counts())


Borough distribution:
Manhattan    2793422
Brooklyn     1220857
Name: start_borough, dtype: int64

End borough distribution:
Manhattan    2788330
Brooklyn     1225949
Name: end_borough, dtype: int64


In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4014279 entries, 0 to 4603571
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  start_borough       object 
 14  end_borough         object 
dtypes: float64(4), object(11)
memory usage: 490.0+ MB


In [ ]:
# Export cleaned dataframe to CSV
df_train.to_csv('data/2024_08/df_train_boroughs.csv', index=False)

In [3]:
df_train_cleaned = pd.read_csv('data/2024_08/df_train_boroughs.csv', low_memory=False)

In [5]:
# Convert datetime columns from string to datetime format
df_train_cleaned['started_at'] = pd.to_datetime(df_train_cleaned['started_at'])
df_train_cleaned['ended_at'] = pd.to_datetime(df_train_cleaned['ended_at'])

df_train_cleaned['trip_duration'] = (df_train_cleaned['ended_at'] - df_train_cleaned['started_at']).dt.total_seconds() / 60

In [6]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4014279 entries, 0 to 4014278
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  start_borough       object        
 14  end_borough         object        
 15  trip_duration       float64       
dtypes: datetime64[ns](2), float64(5), object(9)
memory usage: 490.0+ MB


In [7]:
print(df_train_cleaned['trip_duration'].describe())

count    4.014279e+06
mean     1.370771e+01
std      2.010477e+01
min      1.000333e+00
25%      5.580983e+00
50%      9.673200e+00
75%      1.681856e+01
max      1.498657e+03
Name: trip_duration, dtype: float64


#### Remove outliers based on trip duration

In [9]:
# Calculate IQR
Q1 = df_train_cleaned['trip_duration'].quantile(0.25)
Q3 = df_train_cleaned['trip_duration'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = df_train_cleaned[(df_train_cleaned['trip_duration'] < lower_bound) | 
                    (df_train_cleaned['trip_duration'] > upper_bound)]
print(f"\nNumber of outliers: {len(outliers)} ({len(outliers)/len(df_train_cleaned)*100:.2f}%)")


Number of outliers: 231226 (5.76%)


In [10]:
# Remove outliers using IQR method
df_train_cleaned = df_train_cleaned[(df_train_cleaned['trip_duration'] >= lower_bound) & (df_train_cleaned['trip_duration'] <= upper_bound)]

In [11]:
# Drop rides shorter than 1 minute
df_train_cleaned = df_train_cleaned[df_train_cleaned['trip_duration'] > 1]

In [13]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3783053 entries, 0 to 4014278
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  start_borough       object        
 14  end_borough         object        
 15  trip_duration       float64       
dtypes: datetime64[ns](2), float64(5), object(9)
memory usage: 490.7+ MB


#### Remove non rush-hour times

In [14]:
# Drop- non rush hour times, define rush hour or citibike - 7-10am, 5-7pm only on weekdays (remove August 3rd-4th, 10th-11th, 17th to 18th, 24th and 25th, 31st)

weekend_dates = [
    '2024-08-03', '2024-08-04', 
    '2024-08-10', '2024-08-11',  
    '2024-08-17', '2024-08-18',  
    '2024-08-24', '2024-08-25',  
    '2024-08-31']

rows_to_remove = pd.to_datetime(weekend_dates).date

df_train_cleaned['date'] = df_train_cleaned['started_at'].dt.date

initial_count = len(df_train_cleaned)
df_train_cleaned = df_train_cleaned[~df_train_cleaned['date'].isin(rows_to_remove)]

print(f"Removed {initial_count - len(df_train_cleaned)} rows from specified August weekend dates")
print(f"Remaining rows: {len(df_train_cleaned)}")


Removed 1036456 rows from specified August weekend dates
Remaining rows: 2746597


In [15]:
# Extract hour from started_at
df_train_cleaned['hour'] = df_train_cleaned['started_at'].dt.hour

# Define rush hour times
rush_hours = [7, 8, 9, 17, 18]

# Filter to keep only rush hour rides
df_train_cleaned = df_train_cleaned[df_train_cleaned['hour'].isin(rush_hours)]

# Drop the temporary hour column
df_train_cleaned = df_train_cleaned.drop('hour', axis=1)

print(f"Remaining rows after filtering for rush hours: {len(df_train_cleaned)}")



Remaining rows after filtering for rush hours: 1013116


#### Weather Dataframe

In [17]:
df_weather.head(30)

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,10021,2024-07-31T00:00:00,77.0,77.0,70.1,79.36,0.000,0,NaN,0,...,1013.9,99.8,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
1,10021,2024-07-31T01:00:00,77.0,77.0,70.1,79.40,0.000,0,NaN,0,...,1013.3,89.1,9.9,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
2,10021,2024-07-31T02:00:00,77.0,77.0,71.1,81.90,0.000,0,NaN,0,...,1013.1,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
3,10021,2024-07-31T03:00:00,77.0,77.0,71.1,82.06,0.000,0,NaN,0,...,1012.7,90.0,9.9,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
4,10021,2024-07-31T04:00:00,77.0,77.0,71.1,82.10,0.000,0,NaN,0,...,1012.1,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
5,10021,2024-07-31T05:00:00,76.9,76.9,71.1,82.30,0.000,0,NaN,0,...,1012.2,100.0,9.9,0,0.0,0,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
6,10021,2024-07-31T06:00:00,77.0,77.0,72.0,84.43,0.000,0,NaN,0,...,1012.6,100.0,9.9,15,0.1,0,NaN,Overcast,cloudy,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"
7,10021,2024-07-31T07:00:00,78.1,78.1,71.1,79.21,0.000,0,NaN,0,...,1012.4,89.8,9.9,83,0.3,1,NaN,Partially cloudy,partly-cloudy-day,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"
8,10021,2024-07-31T08:00:00,78.2,78.2,71.1,78.83,0.000,0,NaN,0,...,1012.3,99.3,9.9,104,0.4,1,NaN,Overcast,cloudy,"72502594741,72505394728,KTEB,KLGA,F1417,KNYC,7..."
9,10021,2024-07-31T09:00:00,78.3,78.3,70.1,75.94,0.000,0,NaN,0,...,1012.6,99.3,9.9,139,0.5,1,NaN,Overcast,cloudy,"72505394728,KTEB,KLGA,F1417,KNYC,72503014732"


In [18]:
# Keep only August 2024 data
df_weather = df_weather[df_weather['datetime'].str.contains('2024-08-')]

print(f"Remaining rows: {len(df_weather)}")

Remaining rows: 744


In [19]:
# Drop unnecessary columns from weather data
print(df_weather.columns.tolist())

['name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'conditions', 'icon', 'stations']


In [20]:
df_weather = df_weather.drop(columns=[
    'severerisk',     
    'icon',            
    'stations',        
    'preciptype',      
    'snow',            
    'snowdepth',
    'name',
    'feelslike',        
    'humidity',
    'solarradiation',
    'solarenergy',
    'uvindex',
])

df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 744 entries, 24 to 767
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          744 non-null    object 
 1   temp              744 non-null    float64
 2   dew               744 non-null    float64
 3   precip            744 non-null    float64
 4   precipprob        744 non-null    int64  
 5   windgust          744 non-null    float64
 6   windspeed         744 non-null    float64
 7   winddir           744 non-null    float64
 8   sealevelpressure  744 non-null    float64
 9   cloudcover        744 non-null    float64
 10  visibility        744 non-null    float64
 11  conditions        744 non-null    object 
dtypes: float64(9), int64(1), object(2)
memory usage: 75.6+ KB


In [21]:
# Export cleaned dataframe to CSV
df_train_cleaned.to_csv('data/2024_08/df_train_cleaned.csv', index=False)

### Merge Dataframes

In [22]:
# Sort df_train by the 'started_at' column
df_train_cleaned = df_train_cleaned.sort_values('started_at')
# Round trip duration to nearest 30 seconds
df_train_cleaned['trip_duration'] = (df_train_cleaned['trip_duration'] / .5).round() * .5

In [29]:
# Add column with hour of day to df_train to prepare for merge (rounded to nearest hour)
df_train_cleaned['start_hour'] = df_train_cleaned['started_at'].dt.round('h')

# Add similar column to weather df
df_weather['weather_hour'] = pd.to_datetime(df_weather['datetime']).dt.round('h')

In [32]:
# Merge the dataframes on the hour columns
df_merged = pd.merge(df_train_cleaned, df_weather, left_on='start_hour', right_on='weather_hour', how='left')

print(f"Merged dataframe shape: {df_merged.shape}")
df_merged.head()

Merged dataframe shape: (1013116, 31)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,precip,precipprob,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,conditions,weather_hour
0,B743A57E14D3A694,classic_bike,2024-08-01 07:00:00.170,2024-08-01 07:09:54.156,W 51 St & Rockefeller Plaza,6700.14,West End Ave & W 60 St,7059.08,40.759738,-73.978116,...,0.0,0,9.2,0.0,0.0,1013.8,1.7,9.9,Clear,2024-08-01 07:00:00
1,CA6B198E9A07F6B9,classic_bike,2024-08-01 07:00:01.084,2024-08-01 07:08:32.861,W 54 St & 9 Ave,6920.05,E 51 St & Lexington Ave,6659.01,40.766040,-73.987370,...,0.0,0,9.2,0.0,0.0,1013.8,1.7,9.9,Clear,2024-08-01 07:00:00
2,E6B1133BC8AAAB14,electric_bike,2024-08-01 07:00:02.197,2024-08-01 07:09:58.287,W 31 St & 7 Ave,6331.01,E 23 St & 1 Ave,5929.01,40.749156,-73.991600,...,0.0,0,9.2,0.0,0.0,1013.8,1.7,9.9,Clear,2024-08-01 07:00:00
3,5D97A65D3420F371,electric_bike,2024-08-01 07:00:02.243,2024-08-01 07:07:01.293,West St & Liberty St,5184.08,Church St & Worth St,5359.13,40.711444,-74.014847,...,0.0,0,9.2,0.0,0.0,1013.8,1.7,9.9,Clear,2024-08-01 07:00:00
4,CA27E2C1C6D814F6,electric_bike,2024-08-01 07:00:02.614,2024-08-01 07:08:34.238,E 32 St & Park Ave,6280.12,E 56 St & 3 Ave,6691.11,40.745712,-73.981948,...,0.0,0,9.2,0.0,0.0,1013.8,1.7,9.9,Clear,2024-08-01 07:00:00


In [33]:
# Export the merged dataframe to CSV
df_merged.to_csv('data/2024_08/df_merged.csv', index=False)

In [34]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013116 entries, 0 to 1013115
Data columns (total 31 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             1013116 non-null  object        
 1   rideable_type       1013116 non-null  object        
 2   started_at          1013116 non-null  datetime64[ns]
 3   ended_at            1013116 non-null  datetime64[ns]
 4   start_station_name  1013116 non-null  object        
 5   start_station_id    1013116 non-null  object        
 6   end_station_name    1013116 non-null  object        
 7   end_station_id      1013116 non-null  object        
 8   start_lat           1013116 non-null  float64       
 9   start_lng           1013116 non-null  float64       
 10  end_lat             1013116 non-null  float64       
 11  end_lng             1013116 non-null  float64       
 12  member_casual       1013116 non-null  object        
 13  start_boroug